# Doc2vecの文書ベクトルを使用し、scikit-learnでコサイン類似検索




- Wikipedia＋my-opeの両文書からDoc2Vecモデルを生成


- scikit-learnのcosine-simularityを使用して類似検索

## (1) Wikipediaコンテンツファイルから全文書を抽出

レポート <a href="31-Wikipedia-contents-csv.ipynb"><b>31-Wikipedia-contents-csv.ipynb</b></a> の手順にて、いったんローカルPCにCSVファイル化しておきます。

## (2) Wikipedia文書を学習

レポート <a href="25-Create-doc2vec-model-wiki.ipynb"><b>25-Create-doc2vec-model-wiki.ipynb</b></a> の手順にて生成したDoc2Vecモデルファイルをロードして使用します。

上記手順では、Wikipedia文書、my-ope文書の双方を使用し、ボキャブラリ／単語ベクトルの生成および学習を行い、モデルをファイル保存しています。

In [1]:
'''
    環境準備
'''
import sys
import os

import numpy as np
import pandas as pd
 
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)
if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [2]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec

def doc2vec_model_path(dm):
    model_path = 'prototype/better_algorithm/doc2vec.wiki_myope.PV%d.model' % dm
    return model_path

In [3]:
'''
    あらかじめ学習したモデルのファイルをロード
    dm = 0 : DBoWを使用したモデル
'''
dm = 0
loaded_model_dbow = models.Doc2Vec.load(doc2vec_model_path(dm))

print('Document vector size=%d' % (len(loaded_model_dbow.docvecs)))

Document vector size=87349


## (3) my-opeの文書を、Wikiから生成したモデルにより文書ベクトル化する関数

Wikipedia文書だけで学習されたDoc2Vecモデルを使用し、my-ope文書（質問文）をベクトル化します。

In [4]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 13 # toyotsu_human.csv
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}
learning_parameter = LearningParameter(attr)

In [5]:
from learning.core.nlang import Nlang

In [6]:
def get_document_vector(question, model, warning):
    '''
        question: 
            分かち書きされていない文書
        model:
            Doc2Vecの学習済みモデル
            （検証時は品詞を落としていないWikipedia文書からモデルを生成）

        inferred_vector:
            文書を分かち書きしたコーパスから、
            Doc2Vecの学習済みモデルを使用して
            生成される類似文書ベクトル
            （learning.core.nlang.Nlangの仕様に従い、
            　一部品詞が落とされます。）

        warning:
            Trueを指定時、コーパスに含まれる単語が
            モデル内のWord2Vecボキャブラリにない場合、
            警告を表示する
    '''
    corpus = Nlang.split(question).split()
    inferred_vector = model.infer_vector(corpus, alpha=0.01, min_alpha=0.001, steps=250)
    
    if warning:
        for c in corpus:
            if not c in model.wv.vocab:
                print("Warning: word [%s] does not exist in Word2Vec vocabulary." % c)

    return inferred_vector

def get_document_vectors(questions, model, warning=False):
    document_vectors = []
    for question in questions:
        inferred_vector = get_document_vector(question, model, warning)
        document_vectors.append(list(inferred_vector))

    return np.array(document_vectors)

## (4) コサイン類似検索の実行

質問文は、my-ope プロダクションの nosetests テストケースから引用しました。

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from learning.core.datasource import Datasource
import time

def search_simiarity(question, dbow_model):
    '''
        質問文間でコサイン類似度を算出して、近い質問文の候補を取得する
        
        仕様はプロダクションに準拠しています
        ただし、文書のベクトル化は、TF-IDFではなく、
        Doc2Vecを使用します。
    '''
    start = time.time()

    datasource = Datasource('csv')
    question_answers = datasource.question_answers_for_suggest(_bot_id, question)
    
    '''
        学習時に追加した否定のセットを、ここでもquestion_answersに追加しておきます
        　['id', 'question', 'bot_id', 'answer_id']
    '''
    series = pd.Series([13700, '「退職金」退職金の振込口座を変えたくない', 13, 7050], index=question_answers.columns)
    question_answers = question_answers.append(series, ignore_index=True)
    
    #all_array = TextArray(question_answers['question'], vectorizer=self.vectorizer)
    #question_array = TextArray([question], vectorizer=self.vectorizer)
    all_array      = get_document_vectors(question_answers['question'], dbow_model)
    question_array = get_document_vectors([question], dbow_model, warning=True)
    
    print('count: my-ope all questions=%d, document vectors=%d (features=%d)' % (
        np.size(question_answers['question']), all_array.shape[0], all_array.shape[1]
    ))    
    print('count: question=%d, document vectors=%d (features=%d)' % (
        np.size([question]), question_array.shape[0], question_array.shape[1]
    ))    

    similarities = cosine_similarity(all_array, question_array)
    similarities = similarities.flatten()

    ordered_result = list(map(lambda x: {
        'question_answer_id': float(x[0]), 'similarity': x[1], 'answer_id': x[2]
    }, sorted(zip(question_answers['id'], similarities, question_answers['answer_id']), key=lambda x: x[1], reverse=True)))

    df = pd.DataFrame.from_dict(ordered_result)

    print(df[0:10])
    elapsed_time =  time.time() - start
    print("elapsed %d seconds" % elapsed_time)


In [8]:
# 正解＝6803
search_simiarity('JAL マイレージ', loaded_model_dbow)

count: my-ope all questions=318, document vectors=318 (features=200)
count: question=1, document vectors=1 (features=200)
   answer_id  question_answer_id  similarity
0       6803             13378.0    0.920316
1       6878             13452.0    0.587425
2       6844             13418.0    0.584465
3       6845             13419.0    0.582625
4       6849             13423.0    0.578960
5       6745             13317.0    0.577266
6       6838             13412.0    0.576162
7       6807             13382.0    0.575980
8       6888             13462.0    0.568396
9       6749             13310.0    0.568274
elapsed 2 seconds


In [9]:
# 正解＝6763
search_simiarity('海外の出張費の精算の方法は？', loaded_model_dbow)

count: my-ope all questions=318, document vectors=318 (features=200)
count: question=1, document vectors=1 (features=200)
   answer_id  question_answer_id  similarity
0       6763             13335.0    0.807660
1       6856             13430.0    0.771838
2       6743             13314.0    0.766420
3       6902             13476.0    0.764399
4       6733             13304.0    0.759548
5       6740             13311.0    0.756747
6       6769             13341.0    0.756244
7       6749             13310.0    0.751041
8       6876             13450.0    0.747553
9       6824             13398.0    0.742359
elapsed 2 seconds


In [10]:
# 正解＝6767
search_simiarity('VISAの勘定科目がわからない', loaded_model_dbow) 

count: my-ope all questions=318, document vectors=318 (features=200)
count: question=1, document vectors=1 (features=200)
   answer_id  question_answer_id  similarity
0       6767             13339.0    0.811058
1       6787             13360.0    0.802935
2       6799             13374.0    0.786311
3       6772             13361.0    0.740271
4       6797             13372.0    0.732309
5       6796             13371.0    0.722282
6       6900             13474.0    0.715792
7       6786             13359.0    0.715107
8       6762             13334.0    0.714958
9       6800             13375.0    0.710661
elapsed 2 seconds


In [11]:
# 正解＝6909
search_simiarity('子供が生まれた', loaded_model_dbow) 

count: my-ope all questions=318, document vectors=318 (features=200)
count: question=1, document vectors=1 (features=200)
   answer_id  question_answer_id  similarity
0       6909             13483.0    0.707287
1       7021             13600.0    0.543184
2       6924             13503.0    0.503078
3       6723             13294.0    0.497850
4       6988             13568.0    0.492857
5       6870             13444.0    0.491981
6       6877             13451.0    0.490872
7       6777             13350.0    0.478835
8       6973             13553.0    0.478095
9       7000             13580.0    0.475540
elapsed 2 seconds


### 以下は参考ケース

In [12]:
# 正解＝6975
search_simiarity('退職金の振込口座を変えたい', loaded_model_dbow) 

count: my-ope all questions=318, document vectors=318 (features=200)
count: question=1, document vectors=1 (features=200)
   answer_id  question_answer_id  similarity
0       6975             13555.0    0.990793
1       7050             13700.0    0.965827
2       6997             13577.0    0.795659
3       6826             13400.0    0.733253
4       6918             13497.0    0.688368
5       6955             13535.0    0.683837
6       7000             13580.0    0.678907
7       6954             13534.0    0.669115
8       6958             13538.0    0.666628
9       6964             13544.0    0.664391
elapsed 2 seconds


In [13]:
# 正解＝7050（6975の反対として設定）
search_simiarity('退職金の振込口座を変えたくない', loaded_model_dbow) 

count: my-ope all questions=318, document vectors=318 (features=200)
count: question=1, document vectors=1 (features=200)
   answer_id  question_answer_id  similarity
0       7050             13700.0    0.990555
1       6975             13555.0    0.957918
2       6997             13577.0    0.790626
3       6826             13400.0    0.702125
4       6918             13497.0    0.661871
5       7000             13580.0    0.660801
6       6972             13552.0    0.658294
7       6979             13559.0    0.657246
8       6964             13544.0    0.655657
9       6838             13412.0    0.655330
elapsed 2 seconds


In [14]:
# 正解＝6763（違った言い回し）
search_simiarity('海外出張時の費用を精算したいのですが', loaded_model_dbow)

count: my-ope all questions=318, document vectors=318 (features=200)
count: question=1, document vectors=1 (features=200)
   answer_id  question_answer_id  similarity
0       6856             13430.0    0.796510
1       6802             13377.0    0.795700
2       6795             13370.0    0.780811
3       6790             13364.0    0.769724
4       6824             13398.0    0.765216
5       6902             13476.0    0.760034
6       6873             13447.0    0.758189
7       6762             13334.0    0.754221
8       6763             13335.0    0.751823
9       6887             13461.0    0.751660
elapsed 2 seconds
